In [35]:
#Required Packages
#!pip install tabula-py
#!pip install great_expectations
#!pip install boto3

In [36]:
#Required Imports
from tabula import read_pdf
import great_expectations as ge
import boto3
from io import StringIO

In [52]:
#Two PDF files. Once is valid; the other is not
clean = "https://github.com/AshHimself/etl-pipeline-from-pdf/raw/master/spacex_launch_data.pdf"
messy = "https://github.com/AshHimself/etl-pipeline-from-pdf/raw/master/spacex_launch_messy_data.pdf"

source_pdf = read_pdf(messy, lattice=True, pages="all")
df = source_pdf[0]

In [45]:
df.head()

,Flight Numb,Date,Time (UTC),Booster Ver,iLoanunch Site,Payload,Customer,Mission Outcome
0,1,4/6/10,18:45:00,F9 v1.0 B00,C3CAFS LC-40,Dragon Spacecraft Qualification Unit,SpaceX,Success
1,2,8/12/10,15:43:00,F9 v1.0 B00,C4CAFS LC-40,"Dragon demo flight C1, two CubeSats",NASA (COTS) NRO,Success
2,3,22/5/12,7:44:00,F9 v1.0 B00,C5CAFS LC-40,Dragon demo flight C2+,NASA (COTS),Success
3,4,8/10/12,0:35:00,F9 v1.0 B00,C6CAFS LC-40,SpaceX CRS-1,NASA (CRS),Failure
4,5,1/3/13,15:10:00,F9 v1.0 B00,C7CAFS LC-40,SpaceX CRS-2,NASA (CRS),Success


In [46]:
### Let's cleanup the column names a little bit.

#Rename the dataframe headers

fields = {'Flight Numb': 'flight_number', 'Dr ate': 'date','Time (UTC)': 'time_utc','Booster Ver':'booster_version','iLoanunch Site':'launch_site'}

df = df.rename(columns=fields) #rename columns
df.columns = map(str.lower, df.columns) ##lower case is better


In [47]:
#header names now look a lot better and more ingestiable.
df.head()

,flight_number,date,time_utc,booster_version,launch_site,payload,customer,mission outcome
0,1,4/6/10,18:45:00,F9 v1.0 B00,C3CAFS LC-40,Dragon Spacecraft Qualification Unit,SpaceX,Success
1,2,8/12/10,15:43:00,F9 v1.0 B00,C4CAFS LC-40,"Dragon demo flight C1, two CubeSats",NASA (COTS) NRO,Success
2,3,22/5/12,7:44:00,F9 v1.0 B00,C5CAFS LC-40,Dragon demo flight C2+,NASA (COTS),Success
3,4,8/10/12,0:35:00,F9 v1.0 B00,C6CAFS LC-40,SpaceX CRS-1,NASA (CRS),Failure
4,5,1/3/13,15:10:00,F9 v1.0 B00,C7CAFS LC-40,SpaceX CRS-2,NASA (CRS),Success


In [48]:
#How can we ensure the quality of this data if this process is to be automated? 
#If you have ever had to use a PDF document as a source location before you'll know that its possible that whoever is creating 
# these PDFs might change the field names. Let's test the schema so that if we import this document again on a new document
# it'll break gracfully. 





In [49]:
df_ge = ge.dataset.PandasDataset(df) #Load pd dataframe into great_expectations (GE)

#Define a rule to define the order and fields that should exist
df_ge.expect_table_columns_to_match_ordered_list(['flight_number',
 'date',
 'time_utc',
 'booster_version',
 'launch_site',
 'payload',
 'customer',
 'outcome'])
#flight_number should ALWAYS be unique. We don't want duplicate data in our warehouse
df_ge.expect_column_values_to_be_unique('flight_number')

#flight_number should never be null
df_ge.expect_column_values_to_not_be_null('flight_number')

#flight_number should be an integer
df_ge.expect_column_values_to_be_of_type('flight_number', 'int64')

#Validate our data
validation_results = df_ge.validate()

#If our data is valid validation_results["success"] should return True
if(validation_results["success"] == True):
    print ('All assertions have passed! :)')
else:
    print ('Some assertions have failed! :(')    

Some assertions have failed! :(


In [50]:
AWSAccessKeyId='your_access_key_id'
AWSSecretKey='your_secret_key'


bucket = 'scrape-pdf-example' # your S3 bucket

csv_buffer = StringIO() 

s3 = boto3.client('s3', aws_access_key_id=AWSAccessKeyId, aws_secret_access_key=AWSSecretKey)

df.to_csv(csv_buffer) #save our dataframe into memory
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'df.csv').put(Body=csv_buffer.getvalue()) #push CSV to S3

{'ResponseMetadata': {'RequestId': '246417CF21013C54',
  'HostId': 'eCxxcIzThWnvDaUEkMXOHUiEdcfZ8NtnvzpetgF185vz0yRJ4vaz6JTSX/axlHE0QFFVx15z8yk=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'eCxxcIzThWnvDaUEkMXOHUiEdcfZ8NtnvzpetgF185vz0yRJ4vaz6JTSX/axlHE0QFFVx15z8yk=',
   'x-amz-request-id': '246417CF21013C54',
   'date': 'Fri, 18 Sep 2020 14:36:34 GMT',
   'etag': '"6e98062717aebcb4ee450a074b5e2e8f"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 1},
 'ETag': '"6e98062717aebcb4ee450a074b5e2e8f"'}